In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load the dataset
data = pd.read_excel('time_series_north_extent.xlsx')

# Convert to Numpy array
data['extent'] = np.asarray(data['extent'])
data_np = data['extent']

# Define the GRU model
model = tf.keras.Sequential([
    tf.keras.layers.GRU(32, activation='relu', input_shape=(None, 1)),
    tf.keras.layers.Dense(1)])

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=50)

# Compile the model
model.compile(loss='mae', optimizer=Adam(learning_rate=0.001), metrics=['mae'])

# Training and Test datasets
train_gru, val_gru = train_test_split(data_np, test_size=0.2,shuffle=False)

history = model.fit(data_np[:-1], data_np[1:], epochs=200, batch_size=16,
                    validation_split=0.2, callbacks=[early_stop])
# Evaluate the model
test_loss, test_mae = model.evaluate(val_gru)
print(f'Test loss: {test_loss}, Test mse: {test_mae}')

In [ ]:
# Plotting the loss iteration
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(history.history["mae"], '-o', label="Training Loss")  
ax.plot(history.history["val_mae"], '-r', label="Validation Loss")
ax.legend(loc='center right', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel("Epoch", fontsize=28)
ax.set_ylabel("Loss", fontsize=28)
ax.set_title("Model Loss (MAE)", fontsize=28)

In [ ]:
# Plot predictions and validation values
import plotly.graph_objects as go 

predictions = model.predict(val_gru)
predictions = pd.DataFrame(predictions)
train_gru_plot, val_gru_plot = train_test_split(data, test_size=0.2,
                                                shuffle=False)

fig = go.Figure()
fig.update_layout(width=800,height=600)
fig.add_trace(go.Scatter(x=val_gru_plot['DATE'],y=val_gru_plot['extent'],
                         name='Validation Data'))
fig.add_trace(go.Scatter(x=val_gru_plot['DATE'],y=predictions[0],
                         name='Prediction'))
fig.update_layout(title='Sea Ice Extent Forecast - GRU',xaxis_title='Date',
                  yaxis_title='Extent')
fig.show()


In [ ]:
# Evaluation Metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,
 mean_absolute_percentage_error

print("R2 score: %s" %r2_score(val_gru_plot['extent'],predictions[0]))
print("Mean Absolute Error: %s" %mean_absolute_error(val_gru_plot['extent'],
                                                     predictions[0]))
print("MSE: %s" %mean_squared_error(val_gru_plot['extent'],predictions[0]))
print("RMSE: %s" %mean_squared_error(val_gru_plot['extent'],predictions[0], 
                                     squared=False))
print("Mean Absolute Percentage Error: %s" %mean_absolute_percentage_error(
    val_gru_plot['extent'],predictions[0]))


Normalization

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
data_norm = pd.read_excel('time_series_north_extent.xlsx')

# Normalize data
data_reshape = data_norm['extent'].values.reshape(-1,1)

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform data using the scaler
normalized_data = scaler.fit_transform(data_reshape)
normalized_data_df = pd.DataFrame(normalized_data,columns=['extent'])
data_norm['extent'] = normalized_data_df['extent']

# Convert to Numpy Array
data_norm['extent'] = np.asarray(data_norm['extent'])
data_np_norm = data_norm['extent']

# Define the GRU model
model_norm = tf.keras.Sequential([
    tf.keras.layers.GRU(32, activation='relu', input_shape=(None, 1)),
    tf.keras.layers.Dense(1)
])

# define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=50)

# Compile the model
model_norm.compile(loss='mae', optimizer=Adam(learning_rate=0.001), 
                   metrics=['mae'])

# Training and Test datasets
train_gru_norm, val_gru_norm = train_test_split(data_np_norm,
                                                test_size=0.2,shuffle=False)

history_norm = model_norm.fit(data_np_norm[:-1], data_np_norm[1:], epochs=200, 
                  batch_size=16, validation_split=0.2, callbacks=[early_stop])

# Evaluate the model
test_loss_norm, test_mae_norm = model_norm.evaluate(val_gru_norm)

print(f'Test loss: {test_loss_norm}, Test mse: {test_mae_norm}')

In [ ]:
# Plotting the loss iteration
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(history_norm.history["mae"], '-o', label="Training Loss")  
ax.plot(history_norm.history["val_mae"], '-r', label="Validation Loss")
ax.legend(loc='center right', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel("Epoch", fontsize=28)
ax.set_ylabel("Loss", fontsize=28)
ax.set_title("Model Loss (MAE)", fontsize=28)

In [ ]:
# Plot predictions and validation values
import plotly.graph_objects as go 

predictions_norm = model_norm.predict(val_gru_norm)
predictions_norm = pd.DataFrame(predictions_norm)
train_gru_plot_norm, val_gru_plot_norm = train_test_split(data_norm, 
                                                  test_size=0.2,shuffle=False)

fig = go.Figure()
fig.update_layout(width=800,height=600)
fig.add_trace(go.Scatter(x=val_gru_plot_norm['DATE'],
                         y=val_gru_plot_norm['extent'],name='Validation Data'))
fig.add_trace(go.Scatter(x=val_gru_plot_norm['DATE'],y=predictions_norm[0],
                         name='Prediction'))
fig.update_layout(title='Sea Ice Extent Forecast - GRU',xaxis_title='Date',
                  yaxis_title='Extent')
fig.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,
 mean_absolute_percentage_error

# Evaluation Metrics
print("R2 score: %s" %r2_score(val_gru_plot_norm['extent'],
                               predictions_norm[0]))
print("Mean Absolute Error: %s" %mean_absolute_error(
    val_gru_plot_norm['extent'],predictions_norm[0]))
print("MSE: %s" %mean_squared_error(val_gru_plot_norm['extent'],
                                    predictions_norm[0]))
print("RMSE: %s" %mean_squared_error(val_gru_plot_norm['extent'],
                                     predictions_norm[0], squared=False))
print("Mean Absolute Percentage Error: %s" %mean_absolute_percentage_error(
    val_gru_plot_norm['extent'],predictions_norm[0]))
